<a href="https://colab.research.google.com/github/mille055/aipi590_project1/blob/main/notebooks/Copy_of_order_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl
!pip install xlrd

In [2]:

from google.colab import files
from google.colab import drive
import os
import pandas as pd



In [29]:

# Read the Excel file using pandas
df = pd.read_excel('/content/cleaned_order_list03082024_larger.xlsx', sheet_name='Sheet1', header=0)
#df.drop(columns=['d', 'e', 'f'], inplace=True)
# Print the DataFrame
df.head()


,Procedure,Reason for Exam,Previous Procedure Name,Reason for Exam Full,Creatinine (mg/dL),Dialysis,Contrast Allergy,Allergy Severity,Unnamed: 8,Predicted Procedure,Protocol,Protocol comments
0,CT abdomen pelvis with contrast,"Small cell lung cancer (SCLC), monitor",NaN,"Small cell lung cancer (SCLC), monitor; Small ...",0.8,NaN,NaN,NaN,NaN,CT abdomen pelvis with contrast,routine,split
1,CT abdomen pelvis with contrast,peri-vesical mass,CT abdomen pelvis with and without contrast,peri-vesical mass; Renal mass,0.8,NaN,NaN,NaN,NaN,CT abdomen pelvis with and without contrast,gu,NaN
2,CT abdomen pelvis with contrast,"Non-small cell lung cancer (NSCLC), metastatic...",CT chest abdomen pelvis with contrast w MIPS,"Non-small cell lung cancer (NSCLC), metastatic...",0.8,NaN,NaN,NaN,NaN,CT abdomen pelvis with contrast,routine,split
3,CT abdomen pelvis with contrast,LLQ abdominal pain,NaN,LLQ abdominal pain; Left lower quadrant pain,0.8,NaN,NaN,NaN,NaN,CT abdomen pelvis with contrast,routine,NaN
4,CT abdomen pelvis with contrast,Nausea/vomiting; Epigastric pain,NaN,Nausea/vomiting; Epigastric pain; Epigastric pain,0.8,NaN,NaN,NaN,NaN,CT abdomen pelvis with contrast,routine,NaN


In [30]:
# was for prior version
# # remove initial text before ':'
# import re
# columns_to_clean = ['Reason for Exam', 'Coded Diagnosis', 'Ordering Comments']
# for column in columns_to_clean:
#   df[column] = df[column].str.replace(r'.*:', '', regex=True)
# df.head()


In [31]:
# clean data by removing those without predicted protocol and other cleanup
df1 = df.copy()
df1 = df1.dropna(subset=['Protocol'])
df1 = df1.drop(columns=['Unnamed: 8'])
df1['Protocol comments'] = df1['Protocol comments'].apply(lambda x: None if pd.isnull(x) else x.split(','))
df1 = df1.reset_index(drop=True)
df1.head()
df1

,Procedure,Reason for Exam,Previous Procedure Name,Reason for Exam Full,Creatinine (mg/dL),Dialysis,Contrast Allergy,Allergy Severity,Predicted Procedure,Protocol,Protocol comments
0,CT abdomen pelvis with contrast,"Small cell lung cancer (SCLC), monitor",NaN,"Small cell lung cancer (SCLC), monitor; Small ...",0.8,NaN,NaN,NaN,CT abdomen pelvis with contrast,routine,[split]
1,CT abdomen pelvis with contrast,peri-vesical mass,CT abdomen pelvis with and without contrast,peri-vesical mass; Renal mass,0.8,NaN,NaN,NaN,CT abdomen pelvis with and without contrast,gu,None
2,CT abdomen pelvis with contrast,"Non-small cell lung cancer (NSCLC), metastatic...",CT chest abdomen pelvis with contrast w MIPS,"Non-small cell lung cancer (NSCLC), metastatic...",0.8,NaN,NaN,NaN,CT abdomen pelvis with contrast,routine,[split]
3,CT abdomen pelvis with contrast,LLQ abdominal pain,NaN,LLQ abdominal pain; Left lower quadrant pain,0.8,NaN,NaN,NaN,CT abdomen pelvis with contrast,routine,None
4,CT abdomen pelvis with contrast,Nausea/vomiting; Epigastric pain,NaN,Nausea/vomiting; Epigastric pain; Epigastric pain,0.8,NaN,NaN,NaN,CT abdomen pelvis with contrast,routine,None
...,...,...,...,...,...,...,...,...,...,...,...
1150,CT RCC Protocol Dual Abd W WO Contrast,"CT; Kidney cancer, monitor",CT RCC protocol incl chest w MIPS and dual abd...,"CT; Kidney cancer, monitor; History of renal c...",0.8,NaN,1.0,Mild,CT RCC Protocol Dual Abd W WO Contrast,rcc,[steroid prep]
1151,CT renal stone protocol inc CT abd and pelvis ...,pain,NaN,"pain; Acute right flank pain, unspecified",0.8,NaN,NaN,NaN,CT renal stone protocol inc CT abd and pelvis ...,renal stone,None
1152,CT renal stone protocol inc CT abd and pelvis ...,hematuria,NaN,hematuria; Gross hematuria,1.0,NaN,NaN,NaN,CT renal stone protocol inc CT abd and pelvis ...,renal stone,None
1153,CT renal stone protocol inc CT abd and pelvis ...,flank pain,NaN,"flank pain; Flank pain, acute",1.1,NaN,NaN,NaN,CT renal stone protocol inc CT abd and pelvis ...,renal stone,None


In [34]:
# save to an excel file that can be downloaded
df1.to_excel('cleaned_order_data_large.xlsx', index=False)
files.download('cleaned_order_data_large.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:

with open("/content/opaikey.txt", "r") as f:
  OPENAI_API_KEY = f.read().strip()


In [39]:
!pip install openai --upgrade

In [64]:

import openai
from openai import OpenAI

def summarize_patient_visit(row):
  """
  This function takes a row from the DataFrame and uses the OpenAI API to generate a summary of the last patient visit or clinical vignette.

  Args:
    row: A row from the DataFrame.

  Returns:
    A string containing the summary of the last patient visit or clinical vignette.
  """

  prompt = (
      "Please provide a one to three sentence summary of the last patient visit or clinical vignette for the following patient that briefly provides the back story on why the imaging is being performed. This may include comorbid conditions such as HTN, high blood pressure, cancer history, basic demographics, surgical history, or what the leading diagnosis is, etc:\n\n"

      f"Reason for Exam: {row['Reason for Exam Full']}\n"
      f"Ordered study: {row['Procedure']}\n"


  )
  client = OpenAI(api_key = OPENAI_API_KEY)

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a virtual assistant who is helping a doctor summarize patient visits."},
          {"role": "user", "content": prompt}
      ],
      temperature=0.5,
      max_tokens=100,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=["\n"]
  )

  return response.choices[0].message.content.strip()

df2 = df1.copy()
df2['clinical summary'] = df2.apply(summarize_patient_visit, axis=1)

df2.to_excel('cleaned_order_data_large_summaries.xlsx', index=False)
files.download('cleaned_order_data_large_summaries.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [62]:
df2['clinical summary'][3]

'Patient is a 45-year-old female presenting with left lower quadrant abdominal pain. Given the location and nature of the pain, a CT abdomen pelvis with contrast was ordered to evaluate for possible causes such as diverticulitis, ovarian pathology, or inflammatory bowel disease.'